# Tugas Individu Teknologi Basis Data
- Nama: Muhammad Haerul
- NIM: H071201033
- Kelas: A 

## Create Engine

In [1]:
from sqlalchemy import create_engine
engine = create_engine('mysql://root@localhost/classicmodels', echo=False)

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
Base.metadata.create_all(engine)

from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind = engine)
session = Session()

## Create Models

In [2]:
from sqlalchemy import Column, DECIMAL, Date, ForeignKey, String, Text
from sqlalchemy.dialects.mysql import INTEGER, MEDIUMBLOB, MEDIUMTEXT, SMALLINT
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()
metadata = Base.metadata

class Office(Base):
    __tablename__ = 'offices'

    officeCode = Column(String(10), primary_key=True)
    city = Column(String(50), nullable=False)
    phone = Column(String(50), nullable=False)
    addressLine1 = Column(String(50), nullable=False)
    addressLine2 = Column(String(50))
    state = Column(String(50))
    country = Column(String(50), nullable=False)
    postalCode = Column(String(15), nullable=False)
    territory = Column(String(10), nullable=False)


class Productline(Base):
    __tablename__ = 'productlines'

    productLine = Column(String(50), primary_key=True)
    textDescription = Column(String(4000))
    htmlDescription = Column(MEDIUMTEXT)
    image = Column(MEDIUMBLOB)


class Employee(Base):
    __tablename__ = 'employees'

    employeeNumber = Column(INTEGER(11), primary_key=True)
    lastName = Column(String(50), nullable=False)
    firstName = Column(String(50), nullable=False)
    extension = Column(String(10), nullable=False)
    email = Column(String(100), nullable=False)
    officeCode = Column(ForeignKey('offices.officeCode'), nullable=False, index=True)
    reportsTo = Column(ForeignKey('employees.employeeNumber'), index=True)
    jobTitle = Column(String(50), nullable=False)

    office = relationship('Office')
    parent = relationship('Employee', remote_side=[employeeNumber])


class Product(Base):
    __tablename__ = 'products'

    productCode = Column(String(15), primary_key=True)
    productName = Column(String(70), nullable=False)
    productLine = Column(ForeignKey('productlines.productLine'), nullable=False, index=True)
    productScale = Column(String(10), nullable=False)
    productVendor = Column(String(50), nullable=False)
    productDescription = Column(Text, nullable=False)
    quantityInStock = Column(SMALLINT(6), nullable=False)
    buyPrice = Column(DECIMAL(10, 2), nullable=False)
    MSRP = Column(DECIMAL(10, 2), nullable=False)

    productline = relationship('Productline')


class Customer(Base):
    __tablename__ = 'customers'

    customerNumber = Column(INTEGER(11), primary_key=True)
    customerName = Column(String(50), nullable=False)
    contactLastName = Column(String(50), nullable=False)
    contactFirstName = Column(String(50), nullable=False)
    phone = Column(String(50), nullable=False)
    addressLine1 = Column(String(50), nullable=False)
    addressLine2 = Column(String(50))
    city = Column(String(50), nullable=False)
    state = Column(String(50))
    postalCode = Column(String(15))
    country = Column(String(50), nullable=False)
    salesRepEmployeeNumber = Column(ForeignKey('employees.employeeNumber'), index=True)
    creditLimit = Column(DECIMAL(10, 2))

    employee = relationship('Employee')


class Order(Base):
    __tablename__ = 'orders'

    orderNumber = Column(INTEGER(11), primary_key=True)
    orderDate = Column(Date, nullable=False)
    requiredDate = Column(Date, nullable=False)
    shippedDate = Column(Date)
    status = Column(String(15), nullable=False)
    comments = Column(Text)
    customerNumber = Column(ForeignKey('customers.customerNumber'), nullable=False, index=True)

    customer = relationship('Customer')


class Payment(Base):
    __tablename__ = 'payments'

    customerNumber = Column(ForeignKey('customers.customerNumber'), primary_key=True, nullable=False)
    checkNumber = Column(String(50), primary_key=True, nullable=False)
    paymentDate = Column(Date, nullable=False)
    amount = Column(DECIMAL(10, 2), nullable=False)

    customer = relationship('Customer')


class Orderdetail(Base):
    __tablename__ = 'orderdetails'

    orderNumber = Column(ForeignKey('orders.orderNumber'), primary_key=True, nullable=False)
    productCode = Column(ForeignKey('products.productCode'), primary_key=True, nullable=False, index=True)
    quantityOrdered = Column(INTEGER(11), nullable=False)
    priceEach = Column(DECIMAL(10, 2), nullable=False)
    orderLineNumber = Column(SMALLINT(6), nullable=False)

    order = relationship('Order')
    product = relationship('Product')

## Define and Import Helper Functions

In [10]:
import pandas as pd
from sqlalchemy.sql import func
from sqlalchemy import or_

def to_df_dict(query):
    return pd.DataFrame([i.__dict__ for i in query]).drop(columns='_sa_instance_state')

def to_df(query):
    df = pd.read_sql(query.statement, engine)
    return df

## Questions

### 1. Prepare a list of offices sorted by country, state, city.

In [4]:
o = Office
num1 = session.query(o.officeCode, o.country, o.state, o.city).order_by(Office.country, Office.state, Office.city)

df1 = to_df(num1)
df1.style.hide(axis='index')

officeCode,country,state,city
6,Australia,None,Sydney
4,France,None,Paris
5,Japan,Chiyoda-Ku,Tokyo
7,UK,None,London
1,USA,CA,San Francisco
2,USA,MA,Boston
3,USA,NY,NYC


### 2. How many employees are there in the company?

In [174]:
num2 = session.query(Employee).count()
print('Number of employee =', num2)

Number of employee = 23


### 3. What is the total of payments received?

In [7]:
num3 = session.query(func.sum(Payment.amount)).scalar()
print('Total payment =', num3)

Total payment = 8853839.23


### 4. List the product lines that contain 'Cars'

In [182]:
num4 = session.query(Productline).filter(Productline.productLine.contains('Cars'))

df4 = to_df(num4)
df4.style.hide(axis='index')

productLine,textDescription,htmlDescription,image
Classic Cars,"Attention car enthusiasts: Make your wildest car ownership dreams come true. Whether you are looking for classic muscle cars, dream sports cars or movie-inspired miniatures, you will find great choices in this category. These replicas feature superb attention to detail and craftsmanship and offer features such as working steering system, opening forward compartment, opening rear trunk with removable spare wheel, 4-wheel independent spring suspension, and so on. The models range in size from 1:10 to 1:24 scale and include numerous limited edition and several out-of-production vehicles. All models include a certificate of authenticity from their manufacturers and come fully assembled and ready for display in the home or office.",None,None
Vintage Cars,"Our Vintage Car models realistically portray automobiles produced from the early 1900s through the 1940s. Materials used include Bakelite, diecast, plastic and wood. Most of the replicas are in the 1:18 and 1:24 scale sizes, which provide the optimum in detail and accuracy. Prices range from $30.00 up to $180.00 for some special limited edition replicas. All models include a certificate of authenticity from their manufacturers and come fully assembled and ready for display in the home or office.",None,None


### 5. Report total payments for October 28, 2004.

In [177]:
num5 = session.query(func.sum(Payment.amount)).filter(Payment.paymentDate.contains("2004-10-28")).scalar()
print(f"Total Payments for October 28, 2004 is ${num5}")

Total Payments for October 28, 2004 is $47411.33


### 6. Report those payments greater than $100,000.

In [183]:
num6 = session.query(Payment).filter(Payment.amount > 100000)

df6 = to_df(num6)
df6.style.hide(axis='index')

customerNumber,checkNumber,paymentDate,amount
124,AE215433,2005-03-05,101244.590000
124,KI131716,2003-08-15,111654.400000
141,ID10962,2004-12-31,116208.400000
141,JE105477,2005-03-18,120166.580000
148,KM172879,2003-12-26,105743.000000


### 7. List the products in each product line.

In [184]:
num7 = session.query(Product, func.group_concat(Product.productName)).group_by(Product.productLine)

df7 = to_df(num7).rename(columns={'group_concat_1':'listProduct'})
df7[['productLine', 'listProduct']].style.hide(axis='index')

productLine,listProduct
Classic Cars,"1952 Alpine Renault 1300,1972 Alfa Romeo GTA,1962 LanciaA Delta 16V,1968 Ford Mustang,2001 Ferrari Enzo,1969 Corvair Monza,1968 Dodge Charger,1969 Ford Falcon,1970 Plymouth Hemi Cuda,1969 Dodge Charger,1993 Mazda RX-7,1965 Aston Martin DB5,1948 Porsche 356-A Roadster,1995 Honda Civic,1998 Chrysler Plymouth Prowler,1999 Indy 500 Monte Carlo SS,1992 Ferrari 360 Spider red,1985 Toyota Supra,1969 Dodge Super Bee,1976 Ford Gran Torino,1948 Porsche Type 356 Roadster,1970 Triumph Spitfire,1957 Corvette Convertible,1957 Ford Thunderbird,1970 Chevy Chevelle SS 454,1970 Dodge Coronet,1966 Shelby Cobra 427 S/C,1949 Jaguar XK 120,1958 Chevy Corvette Limited Edition,1952 Citroen-15CV,1982 Lamborghini Diablo,1969 Chevrolet Camaro Z28,1971 Alpine Renault 1600s,2002 Chevy Corvette,1956 Porsche 356A Coupe,1992 Porsche Cayenne Turbo Silver,1961 Chevrolet Impala,1982 Camaro Z28"
Motorcycles,"1969 Harley Davidson Ultimate Chopper,1996 Moto Guzzi 1100i,2003 Harley-Davidson Eagle Drag Bike,2002 Suzuki XREO,1936 Harley Davidson El Knucklehead,1957 Vespa GS150,1997 BMW R 1100 S,1960 BSA Gold Star DBD34,1982 Ducati 900 Monster,1997 BMW F650 ST,1982 Ducati 996 R,1974 Ducati 350 Mk3 Desmo,2002 Yamaha YZR M1"
Planes,"1980s Black Hawk Helicopter,P-51-D Mustang,1928 British Royal Navy Airplane,1900s Vintage Bi-Plane,Corsair F4U ( Bird Cage),1900s Vintage Tri-Plane,American Airlines: B767-300,America West Airlines B757-200,ATA: B757-300,F/A 18 Hornet 1/72,American Airlines: MD-11S,Boeing X-32A JSF"
Ships,"1999 Yamaha Speed Boat,18th century schooner,The Schooner Bluenose,The Mayflower,HMS Bounty,The USS Constitution Ship,The Titanic,The Queen Mary,Pont Yacht"
Trains,"Collectable Wooden Train,1950's Chicago Surface Lines Streetcar,1962 City of Detroit Streetcar"
Trucks and Buses,"1958 Setra Bus,1957 Chevy Pickup,1940 Ford Pickup Truck,1964 Mercedes Tour Bus,1926 Ford Fire Engine,1940s Ford truck,1962 Volkswagen Microbus,1980’s GM Manhattan Express,1954 Greyhound Scenicruiser,1996 Peterbilt 379 Stake Bed with Outrigger,Diamond T620 Semi-Skirted Tanker"
Vintage Cars,"1937 Lincoln Berline,1936 Mercedes-Benz 500K Special Roadster,1917 Grand Touring Sedan,1911 Ford Town Car,1932 Model A Ford J-Coupe,1928 Mercedes-Benz SSK,1913 Ford Model T Speedster,1934 Ford V8 Coupe,18th Century Vintage Horse Carriage,1903 Ford Model A,1917 Maxwell Touring Car,1941 Chevrolet Special Deluxe Cabriolet,1932 Alfa Romeo 8C2300 Spider Sport,1904 Buick Runabout,1939 Cadillac Limousine,1939 Chevrolet Deluxe Coupe,1938 Cadillac V-16 Presidential Limousine,1912 Ford Model T Delivery Wagon,1937 Horch 930V Limousine,1940 Ford Delivery Sedan,1936 Mercedes Benz 500k Roadster,1936 Chrysler Airflow,1928 Ford Phaeton Deluxe,1930 Buick Marquette Phaeton"


### 8. How many products in each product line?

In [185]:
num8 = session.query(Product.productLine, func.count(Product.productCode)).group_by(Product.productLine)

df8 = to_df(num8).rename(columns={'count_1':'countProduct'})
df8.style.hide(axis='index')

productLine,countProduct
Classic Cars,38
Motorcycles,13
Planes,12
Ships,9
Trains,3
Trucks and Buses,11
Vintage Cars,24


### 9. What is the minimum payment received?

In [157]:
num9 = session.query(func.min(Payment.amount)).scalar()
print('Minimum payment =', num9)

Minimum payment = 615.45


### 10. List all payments greater than twice the average payment.

In [186]:
avg_payment = session.query(func.avg(Payment.amount)).scalar()
num10 = session.query(Payment).filter(Payment.amount >= avg_payment*2)

df10 = to_df(num10)
df10.style.hide(axis='index')

customerNumber,checkNumber,paymentDate,amount
114,MA765515,2004-12-15,82261.220000
124,AE215433,2005-03-05,101244.590000
124,BG255406,2004-08-28,85410.870000
124,ET64396,2005-04-16,83598.040000
124,KI131716,2003-08-15,111654.400000
141,ID10962,2004-12-31,116208.400000
141,IN446258,2005-03-25,65071.260000
141,JE105477,2005-03-18,120166.580000
148,KM172879,2003-12-26,105743.000000
167,GN228846,2003-12-03,85024.460000


### 11. What is the average percentage markup of the MSRP on buyPrice?

In [187]:
a = Product.buyPrice
b = Product.MSRP
num11 = session.query(func.avg((b-a)/a)*100).scalar()
print(f"Average percentage markup of the MSRP on buyPrice is {num11}%")

Average percentage markup of the MSRP on buyPrice is 88.7023921713%


### 12. How many distinct products does ClassicModels sell?

In [160]:
num12 = session.query(Product).distinct().count()
print(f"Distinct products: {num12}")

Distinct products: 110


### 13. Report the name and city of customers who don't have sales representatives?

In [188]:
num13 = session.query(Customer).filter(Customer.salesRepEmployeeNumber == None)

df13 = to_df(num13).rename(columns={'salesRepEmployeeNumber':'salesRepresentatives'})
df13[["customerName", "city", 'salesRepresentatives']].style.hide(axis='index')

customerName,city,salesRepresentatives
Havel & Zbyszek Co,Warszawa,None
Porto Imports Co.,Lisboa,None
"Asian Shopping Network, Co",Singapore,None
Natürlich Autos,Cunewalde,None
ANG Resellers,Madrid,None
Messner Shopping Network,Frankfurt,None
"Franken Gifts, Co",München,None
BG&E Collectables,Fribourg,None
Schuyler Imports,Amsterdam,None
Der Hund Imports,Berlin,None


### 14. What are the names of executives with VP or Manager in their title? Use the CONCAT function to combine the employee's first name and last name into a single field for reporting.

In [18]:
num14 = session.query(Employee.employeeNumber, 
                      func.concat(Employee.firstName, " ", Employee.lastName), 
                      Employee.jobTitle).filter(or_(Employee.jobTitle.contains('VP') , 
                                                    Employee.jobTitle.contains('Manager')))

df14 = to_df(num14).rename(columns={'concat_1':'full_name'})
df14.style.hide(axis='index')

employeeNumber,full_name,jobTitle
1056,Mary Patterson,VP Sales
1076,Jeff Firrelli,VP Marketing
1088,William Patterson,Sales Manager (APAC)
1102,Gerard Bondur,Sale Manager (EMEA)
1143,Anthony Bow,Sales Manager (NA)


### 15. Which orders have a value greater than $5,000?

In [17]:
num15 = session.query(Orderdetail.orderNumber, func.concat("$", Orderdetail.priceEach), 
                      Orderdetail.quantityOrdered, 
                      func.concat("$", Orderdetail.priceEach * Orderdetail.quantityOrdered)).filter(Orderdetail.priceEach * Orderdetail.quantityOrdered > 5000)

df15 = to_df(num15).rename(columns={'concat_3':'Value'})
df15

,orderNumber,concat_1,quantityOrdered,Value
0,10103,$214.30,26,$5571.80
1,10103,$119.67,42,$5026.14
2,10105,$127.84,50,$6392.00
3,10105,$205.72,41,$8434.52
4,10105,$117.97,43,$5072.71
...,...,...,...,...
405,10421,$167.06,35,$5847.10
406,10424,$201.44,50,$10072.00
407,10424,$121.64,49,$5960.36
408,10424,$108.50,54,$5859.00
